In [12]:
import cv2
import numpy as np
import random
import os
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from tqdm import tqdm
from tkinter import messagebox


class ImageGenerator:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title('Image Generator')

        # Define and initialize the variables for GUI inputs
        self.resolution = tk.IntVar(value=512)
        self.square_size = tk.IntVar(value=100)
        self.line_width = tk.IntVar(value=10)
        self.line_spacing = tk.IntVar(value=20)
        self.num_images = tk.IntVar(value=100)
        self.direction = tk.StringVar(value='horizontal')
        self.image_quality = tk.StringVar(value='normal')
        self.output_folder = tk.StringVar(value='')

        # Create the GUI elements
        self.folder_label = tk.Label(self.root, text='Output Folder:')
        self.folder_entry = tk.Entry(self.root, textvariable=self.output_folder)
        self.folder_button = tk.Button(self.root, text='Browse', command=self.select_output_folder)

        self.resolution_label = tk.Label(self.root, text='Resolution:')
        self.resolution_entry = tk.Entry(self.root, textvariable=self.resolution)

        self.square_size_label = tk.Label(self.root, text='Square Size:')
        self.square_size_entry = tk.Entry(self.root, textvariable=self.square_size)

        self.line_width_label = tk.Label(self.root, text='Line Width:')
        self.line_width_entry = tk.Entry(self.root, textvariable=self.line_width)

        self.line_spacing_label = tk.Label(self.root, text='Line Spacing:')
        self.line_spacing_entry = tk.Entry(self.root, textvariable=self.line_spacing)

        self.num_images_label = tk.Label(self.root, text='Number of Images:')
        self.num_images_entry = tk.Entry(self.root, textvariable=self.num_images)

        self.direction_label = tk.Label(self.root, text='Direction:')
        self.direction_combobox = ttk.Combobox(self.root, values=['horizontal', 'vertical', '50/50'], textvariable=self.direction)

        self.quality_label = tk.Label(self.root, text='Image Quality:')
        self.quality_combobox = ttk.Combobox(self.root, values=['normal', 'high', 'low'], textvariable=self.image_quality)

        self.generate_button = tk.Button(self.root, text='Generate Images', command=self.launch_generator)

        # Layout the GUI elements using grid layout
        self.folder_label.grid(row=0, column=0, sticky='e')
        self.folder_entry.grid(row=0, column=1, padx=5, pady=5)
        self.folder_button.grid(row=0, column=2, padx=5, pady=5)

        self.resolution_label.grid(row=1, column=0, sticky='e')
        self.resolution_entry.grid(row=1, column=1, padx=5, pady=5)

        self.square_size_label.grid(row=2, column=0, sticky='e')
        self.square_size_entry.grid(row=2, column=1, padx=5, pady=5)

        self.line_width_label.grid(row=3, column=0, sticky='e')
        self.line_width_entry.grid(row=3, column=1, padx=5, pady=5)

        self.line_spacing_label.grid(row=4, column=0, sticky='e')
        self.line_spacing_entry.grid(row=4, column=1, padx=5, pady=5)

        self.num_images_label.grid(row=5, column=0, sticky='e')
        self.num_images_entry.grid(row=5, column=1, padx=5, pady=5)

        self.direction_label.grid(row=6, column=0, sticky='e')
        self.direction_combobox.grid(row=6, column=1, padx=5, pady=5)

        self.quality_label.grid(row=7, column=0, sticky='e')
        self.quality_combobox.grid(row=7, column=1, padx=5, pady=5)

        self.generate_button.grid(row=8, column=0, columnspan=3, padx=5, pady=5)

    def select_output_folder(self):
        folder_selected = filedialog.askdirectory(parent=self.root)
        if folder_selected:
            self.output_folder.set(folder_selected)
            self.folder_entry.delete(0, tk.END)
            self.folder_entry.insert(tk.END, self.output_folder.get())

    def generate_image(self, direction):
        # Generate a single image with the specified line direction

        # Get the selected settings from the GUI
        resolution = self.resolution.get()
        square_size = self.square_size.get()
        line_width = self.line_width.get()
        line_spacing = self.line_spacing.get()

        # Create a blank grayscale image
        image = np.zeros((resolution, resolution), dtype=np.uint8)

        # Generate the lines
        line_color = int(random.normalvariate(128, 32))
        line_positions = list(range(0, resolution, line_spacing + line_width))
        for pos in line_positions:
            # Generate lines based on the specified direction
            if direction == 'horizontal':
                image[pos:pos+line_width, :] = line_color
            elif direction == 'vertical':
                image[:, pos:pos+line_width] = line_color

        # Randomize the placement of the square within the image
        square_x = random.randint(0, resolution - square_size - 1)
        square_y = random.randint(0, resolution - square_size - 1)

        # Generate the square
        square_color = int(random.normalvariate(128, 32))
        image[square_y:square_y+square_size, square_x:square_x+square_size] = square_color

        # Add a border between the square and lines
        border_width = max(line_width, 2)
        image[square_y:square_y+border_width, square_x:square_x+square_size] = 0
        image[square_y+square_size-border_width:square_y+square_size, square_x:square_x+square_size] = 0
        image[square_y:square_y+square_size, square_x:square_x+border_width] = 0
        image[square_y:square_y+square_size, square_x+square_size-border_width:square_x+square_size] = 0

        # Randomize brightness, contrast, and focus using normal distribution
        brightness = int(random.normalvariate(128, 32))
        if self.image_quality.get() == 'high':
            contrast = random.normalvariate(1.5, 0.2)
            focus = random.normalvariate(0.2, 0.05)
        elif self.image_quality.get() == 'low':
            contrast = random.normalvariate(0.5, 0.1)
            focus = random.normalvariate(1.5, 0.2)
        else:
            contrast = random.normalvariate(1.0, 0.2)
            focus = random.normalvariate(0.5, 0.1)

        # Apply brightness, contrast, and focus
        image = cv2.add(image, brightness)
        image = cv2.multiply(image, contrast)
        image = cv2.GaussianBlur(image, (0, 0), focus)

        return image

    def save_image(self, image, folder, filename):
        # Create the folder if it doesn't exist
        os.makedirs(folder, exist_ok=True)

        # Save the image to the specified folder and filename
        filepath = os.path.join(folder, filename)
        cv2.imwrite(filepath, image)

    def launch_generator(self):
        # Generate multiple images based on the selected settings

        output_folder = self.output_folder.get()
        if not output_folder:
            messagebox.showwarning('Warning', 'Please select an output folder.')
            return

        num_images = self.num_images.get()
        num_horizontal = num_images // 2
        num_vertical = num_images - num_horizontal

        progress_bar = tqdm(total=num_images, desc='Generating images', unit='image')

        # Generate images with horizontal lines
        for i in range(num_horizontal):
            image = self.generate_image('horizontal')
            filename = f'image_{i}.png'
            self.save_image(image, output_folder, filename)
            progress_bar.update(1)

        # Generate images with vertical lines
        for i in range(num_horizontal, num_images):
            image = self.generate_image('vertical')
            filename = f'image_{i}.png'
            self.save_image(image, output_folder, filename)
            progress_bar.update(1)

        progress_bar.close()

        messagebox.showinfo('Information', 'Image generation complete.')

        self.root.destroy()

    def run(self):
        self.root.mainloop()


if __name__ == '__main__':
    generator = ImageGenerator()
    generator.run()


Generating images: 100%|███████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 95.20image/s]
